In [1]:
import polars as pl
from scipy.sparse import coo_array, diags
from sksparse.cholmod import cholesky

from benchmarks.utils import mock_snakemake

DF_CUTOFF = 0

if "snakemake" not in globals():  # noqa: F821
    snakemake = mock_snakemake("compute_ptdf")

lines = pl.read_parquet(snakemake.input.lines)
buses = pl.concat([lines["from_bus"], lines["to_bus"]]).unique()
L = len(lines)
N = len(buses)
lines.sort("from_bus")

line_id,from_bus,to_bus,line_rating,reactance
u32,i64,i64,f64,f64
1,1,2410,1.75,0.000219
2238,3,3972,0.64,0.027643
2241,3,3971,0.64,0.027293
2244,3,3976,0.3,0.018766
8013,3,3975,0.8,0.052057
…,…,…,…,…
7535,8861,8862,3.13,0.034818
7618,8863,8864,19.36,0.007541
7597,8865,8866,0.866083,0.125802


In [2]:
# Step 1: Renumber buses such that they go from 1 to N contiguously.
bus_mapping = (
    buses.sort().to_frame().with_row_index(name="new_bus").rename({"from_bus": "bus"})
)
lines = (
    lines.join(bus_mapping.rename({"bus": "from_bus"}), on="from_bus")
    .drop("from_bus")
    .rename({"new_bus": "from_bus"})
    .join(bus_mapping.rename({"bus": "to_bus"}), on="to_bus")
    .drop("to_bus")
    .rename({"new_bus": "to_bus"})
)
lines.sort("from_bus")

line_id,line_rating,reactance,from_bus,to_bus
u32,f64,f64,u32,u32
1,1.75,0.000219,0,1987
2238,0.64,0.027643,1,3090
2241,0.64,0.027293,1,3089
2244,0.3,0.018766,1,3093
8013,0.8,0.052057,1,3092
…,…,…,…,…
7535,3.13,0.034818,6737,6738
7618,19.36,0.007541,6739,6740
7597,0.866083,0.125802,6741,6742


In [3]:
# Step 2: Form an admittance matrix.
lines = lines.with_columns(suscep=1 / lines["reactance"])
off_diagonal = pl.concat(
    [
        lines.select(
            pl.col("from_bus").alias("i"),
            pl.col("to_bus").alias("j"),
            -pl.col("suscep"),
        ),
        lines.select(
            pl.col("to_bus").alias("i"),
            pl.col("from_bus").alias("j"),
            -pl.col("suscep"),
        ),
    ]
)
# TODO double check
diagonal = (
    pl.concat(
        [
            lines.select("suscep", i=pl.col("from_bus")),
            lines.select("suscep", i=pl.col("to_bus")),
        ]
    )
    .group_by("i")
    .agg(pl.sum("suscep"))
    .select("i", pl.col("i").alias("j"), "suscep")
)
assert len(diagonal) == N, "Diagonal entries should match the number of buses."
admittance_matrix_df = pl.concat([off_diagonal, diagonal])
admittance_matrix_df

i,j,suscep
u32,u32,f64
0,1987,-4566.772343
17,1987,-133.59381
1987,1988,-61.96631
1325,1988,-2133.621717
1324,1988,-243.643906
…,…,…
4451,4451,1080.775507
2096,2096,383.272103
4192,4192,210.224532


In [4]:
# Step 3: Drop last row and column (slack bus).
admittance_matrix_df = admittance_matrix_df.filter(
    pl.col("i") < (N - 1), pl.col("j") < (N - 1)
)
admittance_matrix = coo_array(
    (
        admittance_matrix_df["suscep"].to_numpy(),
        (
            admittance_matrix_df["i"].cast(pl.Float64).to_numpy(),
            admittance_matrix_df["j"].cast(pl.Float64).to_numpy(),
        ),
    ),
    shape=(N - 1, N - 1),
)
admittance_matrix

<COOrdinate sparse array of dtype 'float64'
	with 24118 stored elements and shape (6744, 6744)>

In [5]:
# Step 4: Inverse the admittance matrix to get voltage angles.
# We use the sparse Cholesky factorization since, based on my tests, this is much faster than using
# scipy.linalg.inv, scipy.sparse.linalg.inv, scipy.linag.pinv, scipy.sparse.linalg.spsolve
factor = cholesky(admittance_matrix.tocsc())
voltage_angles = factor.inv()
voltage_angles

/tmp/ipykernel_2015709/646270145.py:5: CholmodTypeConversionWarning: array contains 32 bit integers; but 64 bit integers are needed; slowing down due to converting
  voltage_angles = factor.inv()


<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 45481536 stored elements and shape (6744, 6744)>

In [6]:
adjacency_matrix_T = pl.concat(
    [
        lines.select(
            i=pl.col("line_id") - 1,  # Adjusting for zero-based indexing
            j=pl.col("from_bus"),
            val=pl.lit(1),
        ).filter(pl.col("j") < N - 1),  # Exclude slack bus,
        lines.select(
            i=pl.col("line_id") - 1,  # Adjusting for zero-based indexing
            j=pl.col("to_bus"),
            val=pl.lit(-1),
        ).filter(pl.col("j") < N - 1),  # Exclude slack bus
    ]
)
adjacency_matrix_T = coo_array(
    (
        adjacency_matrix_T["val"].to_numpy(),
        (adjacency_matrix_T["i"].to_numpy(), adjacency_matrix_T["j"].to_numpy()),
    ),
    shape=(len(lines), N - 1),
)
susceptance_matrix = diags(lines.sort("line_id")["suscep"].to_numpy())
power_flow = susceptance_matrix @ coo_array(adjacency_matrix_T) @ voltage_angles
power_flow = power_flow.tocoo()
power_flow_df = pl.DataFrame(
    {
        "injection": power_flow.col,
        "line": power_flow.row + 1,
        "factor": power_flow.data,
    }
)
power_flow_df

injection,line,factor
i32,i32,f64
0,1,1.0
6743,2,-0.001003
6742,2,0.000805
6741,2,0.0008
6740,2,0.00078
…,…,…
4,8690,0.000935
3,8690,-0.005642
2,8690,-0.004111


In [7]:
if DF_CUTOFF is not None:
    power_flow_df = power_flow_df.filter(pl.col("factor").abs() > DF_CUTOFF)
power_flow_df

injection,line,factor
i32,i32,f64
0,1,1.0
6743,2,-0.001003
6742,2,0.000805
6741,2,0.0008
6740,2,0.00078
…,…,…
4,8690,0.000935
3,8690,-0.005642
2,8690,-0.004111


In [8]:
# Unmap buses to original IDs.
power_flow_df_unmapped = (
    power_flow_df.join(bus_mapping.rename({"new_bus": "injection"}), on="injection")
    .drop("injection")
    .rename({"bus": "injection"})
)
power_flow_df_unmapped

line,factor,injection
i32,f64,i64
1,1.0,1
2,-0.001003,8867
2,0.000805,8866
2,0.0008,8865
2,0.00078,8864
…,…,…
8690,0.000935,7
8690,-0.005642,6
8690,-0.004111,4


In [9]:
power_flow_df_unmapped.write_parquet(snakemake.output[0])
snakemake.output[0]

'/data/home/machstg/pyoframe/benchmarks/input_data/energy_planning/intermediary_data_inputs/power_transfer_dist_facts.parquet'